<a href="https://colab.research.google.com/github/Julita257/UMwF/blob/Case-3/Case_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import plotly.express as px
import plotly.graph_objs as go

# Definiowanie okresu
start_date = '2024-01-01'
end_date = '2024-06-05'

# Pobranie danych z Yahoo Finance
data = yf.download('AMZN', start=start_date, end=end_date)

# Tworzenie cech
data['Returns'] = data['Close'].pct_change()
data['SMA_10'] = data['Close'].rolling(window=10).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()

# Usunięcie wartości NA
data.dropna(inplace=True)

# Definiowanie etykiet (1 dla kupna, 0 dla sprzedaży)
data['Signal'] = 0
data.loc[data['SMA_10'] > data['SMA_50'], 'Signal'] = 1
data.loc[data['SMA_10'] < data['SMA_50'], 'Signal'] = 0

features = ['Returns', 'SMA_10', 'SMA_50']
X = data[features]
y = data['Signal']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Budowa modelu
model = XGBClassifier()
model.fit(X_train, y_train)

# Optymalizacja hiperparametrów
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Generowanie sygnałów
data['Predicted_Signal'] = best_model.predict(X)

# Definiowanie zwrotów z inwestycji na podstawie sygnałów
data['Strategy_Returns'] = data['Predicted_Signal'].shift(1) * data['Returns']

# Obliczanie skumulowanych zwrotów
data['Cumulative_Strategy_Returns'] = (1 + data['Strategy_Returns']).cumprod()
data['Cumulative_Market_Returns'] = (1 + data['Returns']).cumprod()

# Wizualizacja wyników
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Cumulative_Market_Returns'], mode='lines', name='Market Returns'))
fig.add_trace(go.Scatter(x=data.index, y=data['Cumulative_Strategy_Returns'], mode='lines', name='Strategy Returns'))
fig.update_layout(title='Cumulative Returns: Market vs Strategy', xaxis_title='Date', yaxis_title='Cumulative Returns')
fig.show()


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
